**RDF creation of full-notch creep test (FNCT) data using OntoFNCT**

In this script, data originating from full-notch creep tests performed to selected high-density polyethylene (PE-HD) materials is read in as CSV data. Then, this data is transformed to RDF data by creating triples using the RDFlib library. Therefore, values are allocated to concepts of the Full-notch creep test ontology (OntoFNCT). The OntoFNCT can be found in the open [OntoFNCT GitHub repository](https://github.com/MarkusSchilling/ontoFNCT). It was created on the basis of the PMD Core Ontology ([PMDco](https://w3id.org/pmd/co)), which was developed within the frame of the joint project Platform MaterialDigital ([PMD](https://material-digital.de)).

For data mapping in accordance with the corresponding ontology (OntoFNCT), the OntoFNCT will be parsed. The CSV data is taken from a dataset published openly available on [Zenodo](https://zenodo.org/records/10143352). Furthermore, the primary data, secondary data and metadata addressed in this script is additionally to be found in the corresponding GitHub repository of [OntoFNCT](https://github.com/MarkusSchilling/ontoFNCT).

The RDF data is directly saved in a graph. Subsequently, the graph is serialized and saved to an RDF and a TTL file, respectively.

# Installing and Importing relevant Python packages

In [ ]:
# Installing relevant python packages.

%pip install rdflib
%pip install cchardet

In [3]:
# Import of relevant packages.

from rdflib import Graph, Literal, URIRef, BNode, Namespace, DC
from rdflib.namespace import RDF, RDFS, SKOS, XSD, OWL
import pandas as pd
import io
import urllib.parse
from urllib.request import urlopen
import cchardet
from Module import Functions as f

# Read in full-notch creep test (FNCT) Data

In [4]:
# Read in data that is supposed to be mapped to the concepts in the OntoFNCT ontology.
# The general and agnostic data source is a dataset on full-notch creep tests that is originally published freely available on Zenodo: https://zenodo.org/records/10143352.
# In this script, secondary data and metadata is read in from respective files in the corresponding GitHub repository (https://github.com/MarkusSchilling/ontoFNCT/tree/main/data).

link_FNCT_secondary_data = "https://raw.githubusercontent.com/MarkusSchilling/ontoFNCT/main/data/secondary_data/FNCT_secondary_data.csv"
link_FNCT_metadata = "https://raw.githubusercontent.com/MarkusSchilling/ontoFNCT/main/data/metadata/FNCT_metadata.csv"

with urllib.request.urlopen(link_FNCT_secondary_data) as url:
    bytes_secondarydata = url.read()

res_sd = cchardet.detect(bytes_secondarydata)
encoding_sd = res_sd['encoding']
string_sd = bytes_secondarydata.decode(encoding_sd)
sd = pd.read_csv(io.StringIO(string_sd), sep=';', skiprows=[1])

with urllib.request.urlopen(link_FNCT_metadata) as url:
    bytes_metadata = url.read()

res_md = cchardet.detect(bytes_metadata)
encoding_md = res_md['encoding']
string_md = bytes_metadata.decode(encoding_md)
md = pd.read_csv(io.StringIO(string_md), sep=';', skiprows=[1])

# RDF Graph Creation using RDFlib

In [5]:
# Creating the RDF graph 'g' that will be importing OntoFNCT (see also: https://github.com/MarkusSchilling/ontoFNCT) to include all OntoFNCT concepts into the graph.
# Since the PMDco is imported by OntoFNCT, PMDco (mid-level ontological) concepts will also be included indirectly.

g = Graph()

In [6]:
## Definition of IRI prefixes (namespaces).

base = Namespace("https://w3id.org/ontofnct/") # Namespace of OntoFNCT
g.bind("base", base)
pmdco = Namespace("https://w3id.org/pmd/co/") # Namespace of PMDco ontology
g.bind("pmdco", pmdco)
unit = Namespace("http://qudt.org/vocab/unit/") # Namespace of QUDT ontology (units)
g.bind("unit", unit)
prov = Namespace("http://www.w3.org/ns/prov#") # Namespace of PROV-O ontology
g.bind("prov", prov)
primary_data = Namespace("https://github.com/MarkusSchilling/ontoFNCT/tree/5bac2e737cb3800e0283341428e260f7265611c4/data/primary_data") # Resource of (exemplary) primary FNCT data
g.bind("primary_data", primary_data)
csvw = Namespace("http://www.w3.org/ns/csvw#") # Namespace of CSVW ontology
g.bind("csvw", csvw)
datacite = Namespace("http://purl.org/spar/datacite/") # Namespace of Datacite vocabulary
g.bind("datacite", datacite)
wild = Namespace("http://purl.org/wild/vocab/") # Namespace of WILD vocabulary
g.bind("wild", wild)

# Definition of instances namespace, i.e., instances created from data mapping use the resource namespace
resource = Namespace("https://w3id.org/ontofnct/resource/")
g.bind("resource", resource)

# Turtle style abbreviation for RDF.type.

a = RDF.type

# Ontology is created using resource namespace and imports ontoFNCT.

onto = URIRef(resource)
g.add((onto, a, OWL.Ontology))
g.add((onto, OWL.imports, URIRef(base)))

# Add creators.

markus = URIRef("https://orcid.org/0000-0002-7094-5371")
niklas = URIRef("https://orcid.org/0009-0001-0315-3964")
ute = URIRef("https://orcid.org/0000-0002-4970-7627")
martin = URIRef("https://orcid.org/0000-0001-9753-345X")

g.add((onto, DC.creator, markus))
g.add((markus, RDFS.label, Literal("Markus Schilling", datatype=XSD.string)))
g.add((onto, DC.creator, niklas))
g.add((niklas, RDFS.label, Literal("Niklas Marschall", datatype=XSD.string)))
g.add((onto, DC.creator, ute))
g.add((ute, RDFS.label, Literal("Ute Niebergall", datatype=XSD.string)))
g.add((onto, DC.creator, martin))
g.add((martin, RDFS.label, Literal("Martin Böhning", datatype=XSD.string)))

g.add((onto, DC.title, Literal("Full-notch creep test ontology (OntoFNCT) data mapping example", datatype=XSD.string)))
g.add((onto, OWL.versionInfo, Literal("1.0.0", datatype=XSD.string)))

# Creation of triples.

# Iterate over the full dataset.

for idx, row in sd.iterrows():
  testpiece_id                        = row["Specimen_ID"]
  process_id                          = row["Process_ID"]
  material                            = row["material"]
  width                               = row["Width w"]
  thickness                           = row["Thickness t"]
  notchnom                            = row["Notch depth nominal nn"]
  An                                  = row["Residual fracture surface nominal An"]
  sigman                              = row["Stress nominal ?n"]
  station                             = row["Measurement Station"]
  medium                              = row["Medium"]
  force                               = row["Force"]
  start                               = row["Start of measurement"]
  end                                 = row["End of measurement"]
  tf                                  = row["Time to failure tf"]
  d                                   = row["Final elongation d"]
  AL1                                 = row["Residual fracture surface measured AL1"]
  AL2                                 = row["Residual fracture surface measured AL2"]
  AL                                  = row["Residual fracture surface measured AL"]
  notchmeas                           = row["Notch depth measured nm"]
  sigmaL                              = row["Stress measured ?L"]
  temp                                = row["Test temperature "]
  conditioning                        = row["Conditioning time"]
  leverage                            = row["Leverage ratio"]


  # Creation of triples for all instances; there is one "block" per instance.

  experimentIRI = URIRef(resource + process_id)
  g.add((experimentIRI, a, pmdco.ProcessIdentifier))
  g.add((experimentIRI, pmdco.value, Literal(process_id, datatype=XSD.string)))

  processIRI = URIRef(experimentIRI + "_process")
  g.add((processIRI, a, base.FullNotchCreepTest))
  g.add((processIRI, a, wild.SequentialActivity))
  g.add((processIRI, pmdco.characteristic, experimentIRI))


  # Process description (process centered modeling) using WiLD vocubulary.
  # In this section, the process of full-notch creep testing is subdivided into individual parts / segments, allowing for a more precise process description.

  processModelIRI = URIRef(experimentIRI + "_process_model")
  g.add((processModelIRI, a, wild.WorkflowModel))
  g.add((processModelIRI, wild.hasBehaviour, processIRI))

  # No. 1
  preMeasuringIRI = URIRef(experimentIRI + "_process_preMeasuring")
  g.add((preMeasuringIRI, a, base.MeasuringProcess))
  g.add((preMeasuringIRI, a, wild.Activity))
  g.add((processIRI, base.subordinateProcess, preMeasuringIRI))

  # No. 2
  mountingIRI = URIRef(experimentIRI + "_process_mounting")
  g.add((mountingIRI, a, base.MountingProcess))
  g.add((mountingIRI, a, wild.Activity))
  g.add((preMeasuringIRI, base.nextProcess, mountingIRI))

  # No. 3
  gripsAlignmentIRI = URIRef(experimentIRI + "_process_gripsAlignment")
  g.add((gripsAlignmentIRI, a, base.GripsAligment))
  g.add((gripsAlignmentIRI, a, wild.Activity))
  g.add((mountingIRI, base.nextProcess, gripsAlignmentIRI))

  # No. 4
  FNCTISOIRI = URIRef(experimentIRI + "_process_FNCTISO")
  g.add((FNCTISOIRI, a, base.TensileTest))
  g.add((FNCTISOIRI, a, wild.Activity))
  g.add((gripsAlignmentIRI, base.nextProcess, FNCTISOIRI))

  # No. 5
  demountingIRI = URIRef(experimentIRI + "_process_demounting")
  g.add((demountingIRI, a, base.DemountingProcess))
  g.add((demountingIRI, a, wild.Activity))
  g.add((FNCTISOIRI, base.nextProcess, demountingIRI))

  # No. 6
  postMeasuringIRI = URIRef(experimentIRI + "_process_postMeasuring")
  g.add((postMeasuringIRI, a, base.MeasuringProcess))
  g.add((postMeasuringIRI, a, wild.Activity))
  g.add((demountingIRI, base.nextProcess, postMeasuringIRI))

  # No. 7
  analysisIRI = URIRef(experimentIRI + "_process_analysis")
  g.add((analysisIRI, a, base.AnalysingProcess))
  g.add((analysisIRI, a, wild.Activity))
  g.add((postMeasuringIRI, base.nextProcess, analysisIRI))

  # Blank nodes for process description.

  process_BNode1 = BNode()
  process_BNode2 = BNode()
  process_BNode3 = BNode()
  process_BNode4 = BNode()
  process_BNode5 = BNode()
  process_BNode6 = BNode()
  process_BNode7 = BNode()

  g.add((processIRI, wild.hasChildActivities, process_BNode1))
  g.add((process_BNode1, RDF.first, preMeasuringIRI))
  g.add((process_BNode1, RDF.rest, process_BNode2))
  g.add((process_BNode2, RDF.first, mountingIRI))
  g.add((process_BNode2, RDF.rest, process_BNode3))
  g.add((process_BNode3, RDF.first, gripsAlignmentIRI))
  g.add((process_BNode3, RDF.rest, process_BNode4))
  g.add((process_BNode4, RDF.first, FNCTISOIRI))
  g.add((process_BNode4, RDF.rest, process_BNode5))
  g.add((process_BNode5, RDF.first, demountingIRI))
  g.add((process_BNode5, RDF.rest, process_BNode6))
  g.add((process_BNode6, RDF.first, postMeasuringIRI))
  g.add((process_BNode6, RDF.rest, process_BNode7))
  g.add((process_BNode7, RDF.first, analysisIRI))
  g.add((process_BNode7, RDF.rest, RDF.nil))


  # Test piece as input prior to Full-notch creep test.

  testpieceID_IRI = URIRef(resource + f"_testpiece_name/{testpiece_id}")
  g.add((testpieceID_IRI, a, pmdco.TestPieceName))
  g.add((testpieceID_IRI, a, pmdco.Metadata))
  g.add((testpieceID_IRI, pmdco.value, Literal(testpiece_id, datatype=XSD.string)))

  testpieceIRI = URIRef(resource  + f"_testpiece/{testpiece_id}")
  g.add((testpieceIRI, a, pmdco.TestPiece))
  g.add((testpieceIRI, pmdco.characteristic, testpieceID_IRI))


  # Test piece(s) as output after Full-notch creep test (typically, 2 fractured parts will occur).

  testpieceAfterTest1ID_IRI = URIRef(testpieceID_IRI + "_afterTest_1")
  g.add((testpieceAfterTest1ID_IRI, a, pmdco.TestPieceName))
  g.add((testpieceAfterTest1ID_IRI, a, pmdco.Metadata))
  g.add((testpieceAfterTest1ID_IRI, pmdco.value, Literal(testpiece_id + "_afterTest_1", datatype=XSD.string)))

  testpieceAfterTest1IRI = URIRef(testpieceIRI  + "_testpiece_afterTest_1")
  g.add((testpieceAfterTest1IRI, a, pmdco.TestPiece))
  g.add((testpieceAfterTest1IRI, pmdco.characteristic, testpieceAfterTest1ID_IRI))

  testpieceAfterTest2ID_IRI = URIRef(testpieceID_IRI + "_afterTest_2")
  g.add((testpieceAfterTest2ID_IRI, a, pmdco.TestPieceName))
  g.add((testpieceAfterTest2ID_IRI, a, pmdco.Metadata))
  g.add((testpieceAfterTest2ID_IRI, pmdco.value, Literal(testpiece_id + "_afterTest_2", datatype=XSD.string)))

  testpieceAfterTest2IRI = URIRef(testpieceIRI  + "_testpiece_afterTest_2")
  g.add((testpieceAfterTest2IRI, a, pmdco.TestPiece))
  g.add((testpieceAfterTest2IRI, pmdco.characteristic, testpieceAfterTest2ID_IRI))

  g.add((processIRI, pmdco.input, testpieceIRI))
  g.add((processIRI, pmdco.output, testpieceAfterTest1IRI))
  g.add((processIRI, pmdco.output, testpieceAfterTest2IRI))
  g.add((testpieceIRI, pmdco.composedOf, testpieceAfterTest1IRI))
  g.add((testpieceIRI, pmdco.composedOf, testpieceAfterTest2IRI))

  # The next section contains material / test piece information.
  materialDescriptionIRI = URIRef(experimentIRI + "_materialDescription")
  g.add((materialDescriptionIRI, a, pmdco.materialDesignation))
  g.add((materialDescriptionIRI, a, pmdco.Metadata))
  g.add((materialDescriptionIRI, pmdco.value, Literal(material, datatype=XSD.string)))
  g.add((testpieceIRI, pmdco.characteristic, materialDescriptionIRI))


  # The next section contains primary data.

  # Raw data / dataset.

  datasetIRI = URIRef(resource + testpiece_id + "_dataset")
  g.add((datasetIRI, a, pmdco.PrimaryData))
  g.add((datasetIRI, a, pmdco.Measurement))
  g.add((datasetIRI, a, pmdco.ValueObject))
  g.add((processIRI, pmdco.output, datasetIRI))

  raw_dataIRI = URIRef(resource + testpiece_id + "_csv")
  g.add((raw_dataIRI, a, csvw.Table))
  g.add((raw_dataIRI, a, pmdco.Dataset))
  g.add((raw_dataIRI, DC.title, Literal(f"_process/{process_id}" + f"_testpiece_id/{testpiece_id}", datatype=XSD.string)))
  g.add((raw_dataIRI, csvw.url, Literal(primary_data + f"{process_id}" + ".csv", datatype=XSD.string)))
  g.add((raw_dataIRI, DC.format, Literal("text/csv", datatype=XSD.string)))
  g.add((datasetIRI, pmdco.resource, raw_dataIRI))

  schemaIRI = URIRef(resource + testpiece_id + "_schema")
  g.add((schemaIRI, a, csvw.Schema))
  g.add((raw_dataIRI, csvw.tableSchema, schemaIRI))

  BNode1 = BNode()
  BNode2 = BNode()
  BNode3 = BNode()
  BNode4 = BNode()

  column1 = BNode()
  g.add((column1, a, csvw.Column))
  g.add((column1, a, pmdco.Duration))
  g.add((column1, pmdco.unit, unit.HR))
  g.add((column1, csvw.name, Literal("Duration", datatype=XSD.string)))
  g.add((column1, DC.title, Literal("Duration", datatype=XSD.string)))
  g.add((column1, csvw.datatype, Literal(1, datatype=XSD.float)))
  g.add((column1, csvw.headerCount, Literal(2, datatype=XSD.float)))

  column2 = BNode()
  g.add((column2, a, csvw.Column))
  g.add((column2, a, pmdco.Force))
  g.add((column2, base.unit, unit.N))
  g.add((column2, csvw.name, Literal("Force", datatype=XSD.string)))
  g.add((column2, DC.title, Literal("Force", datatype=XSD.string)))
  g.add((column2, csvw.datatype, Literal(2, datatype=XSD.float)))
  g.add((column2, csvw.headerCount, Literal(2, datatype=XSD.float)))

  column3 = BNode()
  g.add((column3, a, csvw.Column))
  g.add((column3, a, pmdco.Elongation))
  g.add((column3, pmdco.unit, unit.MilliM))
  g.add((column3, csvw.name, Literal("Elongation", datatype=XSD.string)))
  g.add((column3, DC.title, Literal("Elongation", datatype=XSD.string)))
  g.add((column3, csvw.datatype, Literal(3, datatype=XSD.float)))
  g.add((column3, csvw.headerCount, Literal(2, datatype=XSD.float)))

  column4 = BNode()
  g.add((column4, a, csvw.Column))
  g.add((column4, a, pmdco.EnvironmentalTemperature))
  g.add((column4, pmdco.unit, unit.DEG_C))
  g.add((column4, csvw.name, Literal("Temperature", datatype=XSD.string)))
  g.add((column4, DC.title, Literal("Test temperature", datatype=XSD.string)))
  g.add((column4, csvw.datatype, Literal(4, datatype=XSD.float)))
  g.add((column4, csvw.headerCount, Literal(2, datatype=XSD.float)))

  g.add((schemaIRI, csvw.column, BNode1))
  g.add((BNode1, RDF.first, column1))
  g.add((BNode1, RDF.rest, BNode2))
  g.add((BNode2, RDF.first, column2))
  g.add((BNode2, RDF.rest, BNode3))
  g.add((BNode3, RDF.first, column3))
  g.add((BNode3, RDF.rest, BNode4))
  g.add((BNode4, RDF.first, column4))
  g.add((BNode4, RDF.rest, RDF.nil))


  # Rest of primary data.

  widthIRI = URIRef(experimentIRI + "_width_w")
  g.add((widthIRI, a, pmdco.Width))
  g.add((widthIRI, a, pmdco.PrimaryData))
  g.add((widthIRI, a, pmdco.Measurement))
  g.add((widthIRI, pmdco.value, Literal(width, datatype=XSD.float)))
  g.add((widthIRI, pmdco.unit, unit.MilliM))
  g.add((testpieceIRI, pmdco.characteristic, widthIRI))
  g.add((preMeasuringIRI, pmdco.output, widthIRI))

  thicknessIRI = URIRef(experimentIRI + "_thickness_t")
  g.add((thicknessIRI, a, pmdco.Thickness))
  g.add((thicknessIRI, a, pmdco.PrimaryData))
  g.add((thicknessIRI, a, pmdco.Measurement))
  g.add((thicknessIRI, pmdco.value, Literal(thickness, datatype=XSD.float)))
  g.add((thicknessIRI, pmdco.unit, unit.MilliM))
  g.add((testpieceIRI, pmdco.characteristic, thicknessIRI))
  g.add((preMeasuringIRI, pmdco.output, thicknessIRI))

  nominalNotchDepthIRI = URIRef(experimentIRI + "_nominal_notch_depth_nn")
  g.add((nominalNotchDepthIRI, a, base.NotchDepth))
  g.add((nominalNotchDepthIRI, a, pmdco.PrimaryData))
  g.add((nominalNotchDepthIRI, a, pmdco.SetPoint))
  g.add((nominalNotchDepthIRI, pmdco.value, Literal(notchnom, datatype=XSD.float)))
  g.add((nominalNotchDepthIRI, pmdco.unit, unit.MilliM))
  g.add((testpieceIRI, pmdco.characteristic, nominalNotchDepthIRI))

  nominalLigamentAreaIRI = URIRef(experimentIRI + "_nominal_ligament_area_An")
  g.add((nominalLigamentAreaIRI, a, base.NominalLigamentArea))
  g.add((nominalLigamentAreaIRI, a, pmdco.PrimaryData))
  g.add((nominalLigamentAreaIRI, a, pmdco.Measurement))
  g.add((nominalLigamentAreaIRI, pmdco.value, Literal(An, datatype=XSD.float)))
  g.add((nominalLigamentAreaIRI, pmdco.unit, unit.MilliM2))
  g.add((testpieceIRI, pmdco.characteristic, nominalLigamentAreaIRI))
  g.add((preMeasuringIRI, pmdco.output, nominalLigamentAreaIRI))

  nominalStressIRI = URIRef(experimentIRI + "_nominal_stress_sigma_n")
  g.add((nominalStressIRI, a, base.NominalTensileStress))
  g.add((nominalStressIRI, a, pmdco.PrimaryData))
  g.add((nominalStressIRI, a, pmdco.Measurement))
  g.add((nominalStressIRI, pmdco.value, Literal(sigman, datatype=XSD.float)))
  g.add((nominalStressIRI, pmdco.unit, unit.MegaPa))
  g.add((processIRI, pmdco.characteristic, nominalStressIRI))
  g.add((FNCTISOIRI, pmdco.characteristic, nominalStressIRI))

  forceIRI = URIRef(experimentIRI + "_test_force_F")
  g.add((forceIRI, a, pmdco.Force))
  g.add((forceIRI, a, pmdco.PrimaryData))
  g.add((forceIRI, a, pmdco.SetPoint))
  g.add((forceIRI, pmdco.value, Literal(force, datatype=XSD.float)))
  g.add((forceIRI, pmdco.unit, unit.N))
  g.add((processIRI, pmdco.characteristic, forceIRI))
  g.add((FNCTISOIRI, pmdco.characteristic, forceIRI))


  # The next section contains secondary data.

  tfIRI = URIRef(experimentIRI + "_time_to_failure_tf")
  g.add((tfIRI, a, base.TimeToFailure))
  g.add((tfIRI, a, pmdco.SecondaryData))
  g.add((tfIRI, a, pmdco.Measurement))
  g.add((tfIRI, pmdco.value, Literal(tf, datatype=XSD.float)))
  g.add((tfIRI, pmdco.unit, unit.HR))
  g.add((testpieceIRI, pmdco.characteristic, tfIRI))
  g.add((processIRI, pmdco.output, tfIRI))
  g.add((FNCTISOIRI, pmdco.output, tfIRI))

  dIRI = URIRef(experimentIRI + "_final_elongation_d")
  g.add((dIRI, a, base.FinalElongation))
  g.add((dIRI, a, pmdco.SecondaryData))
  g.add((dIRI, a, pmdco.Measurement))
  g.add((dIRI, pmdco.value, Literal(d, datatype=XSD.float)))
  g.add((dIRI, pmdco.unit, unit.MilliM))
  g.add((testpieceIRI, pmdco.characteristic, dIRI))
  g.add((processIRI, pmdco.output, dIRI))
  g.add((FNCTISOIRI, pmdco.output, dIRI))

  AL1IRI = URIRef(experimentIRI + "_measured_ligament_area_AL1")
  g.add((AL1IRI, a, base.MeasuredLigamentArea))
  g.add((AL1IRI, a, pmdco.SecondaryData))
  g.add((AL1IRI, a, pmdco.Measurement))
  g.add((AL1IRI, pmdco.value, Literal(AL1, datatype=XSD.float)))
  g.add((AL1IRI, pmdco.unit, unit.MilliM))
  g.add((testpieceIRI, pmdco.characteristic, AL1IRI))
  g.add((processIRI, pmdco.output, AL1IRI))
  g.add((postMeasuringIRI, pmdco.output, AL1IRI))

  AL2IRI = URIRef(experimentIRI + "_measured_ligament_area_AL2")
  g.add((AL2IRI, a, base.MeasuredLigamentArea))
  g.add((AL2IRI, a, pmdco.SecondaryData))
  g.add((AL2IRI, a, pmdco.Measurement))
  g.add((AL2IRI, pmdco.value, Literal(AL2, datatype=XSD.float)))
  g.add((AL2IRI, pmdco.unit, unit.MilliM))
  g.add((testpieceIRI, pmdco.characteristic, AL2IRI))
  g.add((processIRI, pmdco.output, AL2IRI))
  g.add((postMeasuringIRI, pmdco.output, AL2IRI))

  ALIRI = URIRef(experimentIRI + "_measured_ligament_area_AL")
  g.add((ALIRI, a, base.MeasuredLigamentArea))
  g.add((ALIRI, a, pmdco.SecondaryData))
  g.add((ALIRI, a, pmdco.Measurement))
  g.add((ALIRI, pmdco.value, Literal(AL, datatype=XSD.float)))
  g.add((ALIRI, pmdco.unit, unit.MilliM))
  g.add((testpieceIRI, pmdco.characteristic, ALIRI))
  g.add((processIRI, pmdco.output, ALIRI))
  g.add((postMeasuringIRI, pmdco.output, ALIRI))

  measuredNotchDepthIRI = URIRef(experimentIRI + "_measured_notch_depth_nm")
  g.add((measuredNotchDepthIRI, a, base.NotchDepth))
  g.add((measuredNotchDepthIRI, a, pmdco.SecondaryData))
  g.add((measuredNotchDepthIRI, a, pmdco.Measurement))
  g.add((measuredNotchDepthIRI, pmdco.value, Literal(notchmeas, datatype=XSD.float)))
  g.add((measuredNotchDepthIRI, pmdco.unit, unit.MilliM))
  g.add((testpieceIRI, pmdco.characteristic, measuredNotchDepthIRI))
  g.add((processIRI, pmdco.output, measuredNotchDepthIRI))
  g.add((postMeasuringIRI, pmdco.output, measuredNotchDepthIRI))

  measuredStressIRI = URIRef(experimentIRI + "_measured_stress_sigma_L")
  g.add((measuredStressIRI, a, base.NominalTensileStress))
  g.add((measuredStressIRI, a, pmdco.SecondaryData))
  g.add((measuredStressIRI, a, pmdco.Measurement))
  g.add((measuredStressIRI, pmdco.value, Literal(sigmaL, datatype=XSD.float)))
  g.add((measuredStressIRI, pmdco.unit, unit.MegaPa))
  g.add((testpieceIRI, pmdco.characteristic, measuredStressIRI))
  g.add((processIRI, pmdco.output, measuredStressIRI))
  g.add((postMeasuringIRI, pmdco.output, measuredStressIRI))


  # The next section contains metadata concerning the process.

  startTimeIRI = URIRef(experimentIRI + "_start_time")
  g.add((startTimeIRI, a, pmdco.StartTime))
  g.add((startTimeIRI, a, pmdco.Metadata))
  g.add((startTimeIRI, a, pmdco.Measurement))
  g.add((startTimeIRI, pmdco.value, Literal(start)))
  g.add((processIRI, pmdco.characteristic, startTimeIRI))
  g.add((FNCTISOIRI, pmdco.characteristic, startTimeIRI))

  endTimeIRI = URIRef(experimentIRI + "_end_time")
  g.add((endTimeIRI, a, pmdco.EndTime))
  g.add((endTimeIRI, a, pmdco.Metadata))
  g.add((endTimeIRI, a, pmdco.Measurement))
  g.add((endTimeIRI, pmdco.value, Literal(end)))
  g.add((processIRI, pmdco.characteristic, endTimeIRI))
  g.add((FNCTISOIRI, pmdco.characteristic, endTimeIRI))

  testContainerIRI = URIRef(experimentIRI + "_test_container")
  g.add((testContainerIRI, a, base.TestContainer))
  g.add((testContainerIRI, a, pmdco.Metadata))
  g.add((testContainerIRI, pmdco.value, Literal(station, datatype=XSD.int)))
  g.add((processIRI, pmdco.characteristic, testContainerIRI))
  g.add((FNCTISOIRI, pmdco.characteristic, testContainerIRI))

  mediumIRI = URIRef(experimentIRI + "_test_medium")
  g.add((mediumIRI, a, pmdco.Medium))
  g.add((mediumIRI, a, pmdco.Metadata))
  g.add((mediumIRI, pmdco.value, Literal(medium, datatype=XSD.string)))
  g.add((processIRI, pmdco.characteristic, mediumIRI))
  g.add((FNCTISOIRI, pmdco.characteristic, mediumIRI))

  temperatureIRI = URIRef(experimentIRI + "_test_temperature")
  g.add((temperatureIRI, a, pmdco.EnvironmentalTemperature))
  g.add((temperatureIRI, a, pmdco.Metadata))
  g.add((temperatureIRI, a, pmdco.SetPoint))
  g.add((temperatureIRI, pmdco.value, Literal(temp, datatype=XSD.float)))
  g.add((temperatureIRI, pmdco.unit, unit.DEG_C))
  g.add((processIRI, pmdco.characteristic, temperatureIRI))
  g.add((FNCTISOIRI, pmdco.characteristic, temperatureIRI))

  conditioningIRI = URIRef(experimentIRI + "_conditioning_time")
  g.add((conditioningIRI, a, base.ConditioningTime))
  g.add((conditioningIRI, a, pmdco.Metadata))
  g.add((conditioningIRI, a, pmdco.SetPoint))
  g.add((conditioningIRI, pmdco.value, Literal(conditioning)))
  g.add((conditioningIRI, pmdco.unit, unit.HR))
  g.add((processIRI, pmdco.characteristic, conditioningIRI))
  g.add((FNCTISOIRI, pmdco.characteristic, conditioningIRI))

  leverArmRatioIRI = URIRef(experimentIRI + "_lever_arm_ratio")
  g.add((leverArmRatioIRI, a, base.LeverArmRatio))
  g.add((leverArmRatioIRI, a, pmdco.Metadata))
  g.add((leverArmRatioIRI, pmdco.value, Literal(leverage, datatype=XSD.int)))
  g.add((processIRI, pmdco.characteristic, leverArmRatioIRI))
  g.add((FNCTISOIRI, pmdco.characteristic, leverArmRatioIRI))

  projectIRI = URIRef(experimentIRI + "_project")
  g.add((projectIRI, a, pmdco.Project))
  g.add((projectIRI, a, pmdco.Metadata))
  g.add((processIRI, pmdco.characteristic, projectIRI))

  standardIRI = URIRef(resource + "_standard")
  g.add((standardIRI, a, pmdco.Norm))
  g.add((standardIRI, a, pmdco.Metadata))
  g.add((standardIRI, pmdco.value, Literal(md["Standard"], datatype=XSD.string)))
  g.add((processIRI, pmdco.characteristic, standardIRI))

  # The next section contains metadata on location, address, institute, etc.

  institutionIRI = URIRef(resource + "_institute")
  g.add((institutionIRI, a, prov.Organization))
  g.add((institutionIRI, a, pmdco.Metadata))
  g.add((institutionIRI, pmdco.value, Literal(md["Institute"], datatype=XSD.string)))
  g.add((processIRI, pmdco.characteristic, institutionIRI))

  locationIRI = URIRef(resource + "_location")
  g.add((locationIRI, a, pmdco.Location))
  g.add((locationIRI, a, pmdco.Metadata))
  g.add((locationIRI, pmdco.value, Literal(md["Location"], datatype=XSD.string)))
  g.add((processIRI, pmdco.characteristic, locationIRI))

  addressIRI = URIRef(resource + "_institute_address")
  g.add((addressIRI, a, pmdco.Address))
  g.add((addressIRI, a, pmdco.Metadata))
  g.add((addressIRI, pmdco.value, Literal(md["Address"], datatype=XSD.string)))
  g.add((processIRI, pmdco.characteristic, addressIRI))

  laboratoryIRI = URIRef(resource + "_laboratory")
  g.add((laboratoryIRI, a, pmdco.Laboratory))
  g.add((laboratoryIRI, a, pmdco.Metadata))
  g.add((laboratoryIRI, pmdco.value, Literal(md["Laboratory"], datatype=XSD.string)))
  g.add((processIRI, pmdco.characteristic, laboratoryIRI))

  # The next section contains metadata concerning machines, components (processing nodes) used.

  machineIRI = URIRef(experimentIRI + "_machine")
  g.add((machineIRI, a, base.FullNotchCreepTestDevice))
  g.add((machineIRI, a, pmdco.Metadata))
  g.add((processIRI, pmdco.executedBy, machineIRI))
  g.add((FNCTISOIRI, pmdco.executedBy, machineIRI))

  machineNameIRI = URIRef(experimentIRI + "_machine_name")
  g.add((machineNameIRI, a, pmdco.NodeName))
  g.add((machineNameIRI, a, pmdco.Metadata))
  g.add((machineNameIRI, pmdco.value, Literal(md["Test machine name"], datatype=XSD.string)))
  g.add((machineIRI, pmdco.characteristic, machineNameIRI))

  machineManufacturerIRI = URIRef(experimentIRI + "_machine_manufacturer")
  g.add((machineManufacturerIRI, a, pmdco.Manufacturer))
  g.add((machineManufacturerIRI, a, pmdco.Metadata))
  g.add((machineManufacturerIRI, pmdco.value, Literal(md["Test machine manufacturer"], datatype=XSD.string)))
  g.add((machineIRI, pmdco.characteristic, machineManufacturerIRI))

  machineManufacturerLocationIRI = URIRef(experimentIRI + "_machine_manufacturer_location")
  g.add((machineManufacturerLocationIRI, a, pmdco.Location))
  g.add((machineManufacturerLocationIRI, a, pmdco.Metadata))
  g.add((machineManufacturerLocationIRI, pmdco.value, Literal(md["Test machine manufacturer Location"], datatype=XSD.string)))
  g.add((machineManufacturerIRI, pmdco.characteristic, machineManufacturerLocationIRI))

  machineSerialNumberIRI = URIRef(experimentIRI + "_machine_serial_number")
  g.add((machineSerialNumberIRI, a, pmdco.NodeSerialNumber))
  g.add((machineSerialNumberIRI, a, pmdco.Metadata))
  g.add((machineSerialNumberIRI, pmdco.value, Literal(md["Test machine serial number"], datatype=XSD.string)))
  g.add((machineIRI, pmdco.characteristic, machineSerialNumberIRI))

  loadCellIRI = URIRef(experimentIRI + "_load_cell")
  g.add((loadCellIRI, a, pmdco.LoadCell))
  g.add((loadCellIRI, a, pmdco.Metadata))
  g.add((processIRI, pmdco.executedBy, loadCellIRI))

  loadCellMaxForceIRI = URIRef(experimentIRI + "_load_cell_maximum_force")
  g.add((loadCellMaxForceIRI, a, pmdco.LoadCellCapacity))
  g.add((loadCellMaxForceIRI, a, pmdco.Metadata))
  g.add((loadCellMaxForceIRI, pmdco.value, Literal(md["Load Cell Maximum Force"][0], datatype=XSD.float)))
  g.add((loadCellMaxForceIRI, pmdco.unit, unit.N))
  g.add((loadCellIRI, pmdco.characteristic, loadCellMaxForceIRI))

  preloadIRI = URIRef(experimentIRI + "_preload")
  g.add((preloadIRI, a, base.Preload))
  g.add((preloadIRI, a, pmdco.Metadata))
  g.add((preloadIRI, pmdco.value, Literal(md["Preload"][0], datatype=XSD.float)))
  g.add((preloadIRI, pmdco.unit, unit.N))
  g.add((processIRI, pmdco.characteristic, preloadIRI))
  g.add((FNCTISOIRI, pmdco.characteristic, preloadIRI))

  preloadTimeIRI = URIRef(experimentIRI + "_preload_time")
  g.add((preloadTimeIRI, a, base.PreloadTime))
  g.add((preloadTimeIRI, a, pmdco.Metadata))
  g.add((preloadTimeIRI, pmdco.value, Literal(md["Preload time"][0], datatype=XSD.float)))
  g.add((preloadTimeIRI, pmdco.unit, unit.SEC))
  g.add((processIRI, pmdco.characteristic, preloadTimeIRI))
  g.add((FNCTISOIRI, pmdco.characteristic, preloadTimeIRI))

  testForceIncreaseRateIRI = URIRef(experimentIRI + "_test_force_increase_rate")
  g.add((testForceIncreaseRateIRI, a, base.TestForceIncreaseRate))
  g.add((testForceIncreaseRateIRI, a, pmdco.Metadata))
  g.add((testForceIncreaseRateIRI, pmdco.value, Literal(md["Test Force increase rate"][0], datatype=XSD.float)))
  g.add((testForceIncreaseRateIRI, pmdco.unit, unit.W))
  g.add((processIRI, pmdco.characteristic, testForceIncreaseRateIRI))
  g.add((FNCTISOIRI, pmdco.characteristic, testForceIncreaseRateIRI))

  extensometerIRI = URIRef(experimentIRI + "_extensometer")
  g.add((machineIRI, a, pmdco.Extensometer))
  g.add((machineIRI, a, pmdco.Metadata))
  g.add((machineIRI, pmdco.characteristic, extensometerIRI))

  extensometerCapacityIRI = URIRef(experimentIRI + "_extensometer_capacity")
  g.add((extensometerCapacityIRI, a, base.ExtensometerCapacity))
  g.add((extensometerCapacityIRI, a, pmdco.Metadata))
  g.add((extensometerCapacityIRI, pmdco.value, Literal(md["Limit Test Movement"][0], datatype=XSD.float)))
  g.add((extensometerIRI, pmdco.characteristic, extensometerCapacityIRI))

  # The following section contains some 'bibliographic' information (metadata) using DataCite.
  # There is a projectIRI defined already (see above).

  funder_IRI = URIRef(experimentIRI + "_funding_party")
  g.add((funder_IRI, a, datacite.OrganizationIdentifier))
  g.add((funder_IRI, datacite.usesIdentifierScheme, datacite.Wikidata))
  g.add((datacite.Wikidata, a, datacite.IdentifierScheme))
  g.add((funder_IRI, pmdco.value, Literal(md["Funding Party"], datatype=XSD.string)))
  g.add((projectIRI, datacite.hasIdentifier, funder_IRI))

  funderID_IRI = URIRef(experimentIRI + "_funding_party_ID")
  g.add((funderID_IRI, a, datacite.OrganizationIdentifier))
  g.add((funderID_IRI, pmdco.value, Literal(md["Funding Party ID"], datatype=XSD.string)))
  g.add((funder_IRI, datacite.hasIdentifier, funderID_IRI))

  LocalFunderIdentifierSchemeIRI = URIRef(datacite + "_local-funder-identifier-scheme")
  FunderRefIRI = URIRef(experimentIRI + "Funder_Identifier")
  g.add((FunderRefIRI, a, datacite.FunderIdentifier))
  g.add((FunderRefIRI, datacite.usesIdentifierScheme, LocalFunderIdentifierSchemeIRI))
  g.add((LocalFunderIdentifierSchemeIRI, a, datacite.FunderIdentifierScheme))
  g.add((datacite.FunderIdentifierScheme, RDFS.subClassOf, datacite.IdentifierScheme))
  g.add((FunderRefIRI, pmdco.value, Literal(md["Grant number"], datatype=XSD.string)))
  g.add((projectIRI, datacite.hasIdentifier, FunderRefIRI))

# Graph Serialization

In [ ]:
# Serialization of RDF graph 'g' performed in two selected data formats (RDF and TTL).

g.serialize("ontoFNCT_exemplary_data_PE-HD.rdf", format='application/rdf+xml', encoding="utf-8")
g.serialize("ontoFNCT_exemplary_data_PE-HD.ttl", format='turtle')